# New cross-model distance

In [3]:
from koselleck import *

In [4]:
cdist('journeys','1750-1755',neighbors=['virtue','truth'], num_proc=1)

Measuring cosine distances across runs [x1]: 100%|██████████| 10/10 [00:00<00:00, 10.99it/s]


cdist
word     neighbor period    run          
journeys virtue   1750-1755 1    1.254887
         truth    1750-1755 1    1.311167
         virtue   1750-1755 2    1.104931
         truth    1750-1755 2    1.203024
         virtue   1750-1755 3    1.222848
         truth    1750-1755 3    1.287891
                            4    1.152832
         virtue   1750-1755 4    1.203968
                            5    1.120570
         truth    1750-1755 5    1.247197
         virtue   1750-1755 6    1.059626
         truth    1750-1755 6    1.073174
         virtue   1750-1755 7    1.208020
         truth    1750-1755 7    1.360738
         virtue   1750-1755 8    1.104199
         truth    1750-1755 8    1.226013
         virtue   1750-1755 9    1.301294
         truth    1750-1755 9    1.304126
         virtue   1750-1755 10   1.172972
         truth    1750-1755 10   1.306250

In [45]:
for x,y in zip([1,2,3],['a','b']):
    print(x,y)

1 a
2 b


In [40]:
# fastdist.cosine_matrix_to_matrix??

In [55]:
def to_lnm(word1_period1,word2_period2,k=25,num_proc=1,cache=True,progress=True):
    
    word1,period1=word1_period1.split('_') if type(word1_period1)==str else word1_period1
    word2,period2=word2_period2.split('_') if type(word2_period2)==str else word2_period2

    qstr=f'{word1}_{period1},{word2}_{period2},k={k}'
    if cache:
        with get_veclib('lnm') as vl:
            if qstr in vl:
                return vl[qstr]
    
    dfnbr1=to_nbr(word1,period1,num_proc=num_proc,progress=progress)
    dfnbr2=to_nbr(word2,period2,num_proc=num_proc,progress=progress)
    
    # filter words in both models
    dfmetanbr=dfnbr1.iloc[:k].append(dfnbr2.iloc[:k])
    metaneighb=list(set(dfmetanbr.index.get_level_values('neighbor')))
    
    # dists
    dfcdist1=cdist(word1,period1,neighbors=metaneighb,progress=progress,num_proc=num_proc).reset_index()
    dfcdist2=cdist(word2,period2,neighbors=metaneighb,progress=progress,num_proc=num_proc).reset_index()
    
    dfcdist1['word_period_run']=list(zip(dfcdist1.word,dfcdist1.period,dfcdist1.run))
    dfcdist2['word_period_run']=list(zip(dfcdist2.word,dfcdist2.period,dfcdist2.run))
    
    dfpiv1=dfcdist1.pivot('word_period_run','neighbor','cdist')
    dfpiv2=dfcdist2.pivot('word_period_run','neighbor','cdist')
    
    dfpiv_dists = cdist_dfpivs(dfpiv1,dfpiv2)
    o=[]
    for i,row in dfpiv_dists.iterrows():
        odx={
            'word1':row.i1[0],
            'word2':row.i2[0],
            'period1':row.i1[1],
            'period2':row.i2[1],
            'run1':row.i1[2],
            'run2':row.i2[2],
            'lnm':row.cdist,
            'mneighb_size':row['size'],
            'neighb1_size':row.size1,
            'neighb2_size':row.size2,
        }
        o+=[odx]
    odf=pd.DataFrame(o)
    index_cols=['word1','word2','period1','period2','run1','run2']
    odf=odf.set_index(index_cols)
    
    if cache:
        with get_veclib('lnm',autocommit=True) as vl:
            vl[qstr]=odf
    
    return odf

In [56]:
odf=to_lnm('culture_1750-1755','culture_1850-1855')
odf

Measuring cosine distances across runs [x1]: 100%|██████████| 10/10 [00:00<00:00, 21.29it/s]


lnm  mneighb_size  \
word1   word2   period1   period2   run1 run2                           
culture culture 1750-1755 1850-1855 1    1     0.154668            43   
                                    2    2     0.141473            42   
                                    3    3     0.185739            43   
                                    4    4     0.137825            43   
                                    5    5     0.152937            43   
                                    6    6     0.120600            42   
                                    7    7     0.189098            43   
                                    8    8     0.097184            44   
                                    9    9     0.132858            43   
                                    10   10    0.199079            44   

                                               neighb1_size  neighb2_size  
word1   word2   period1   period2   run1 run2                              
culture culture 1750-1755 1850-1855 1    1               44            48  
                                    2    2               43            48  
                                    3    3               44            48  
                                    4    4               44            48  
                                    5    5               44            48  
                                    6    6               43            48  
                                    7    7               44            48  
                                    8    8               45            48  
                                    9    9               44            48  
                                    10   10              45            48

In [57]:
# odf.groupby(['word1','word2','period1','period2']).mean()

In [58]:
# odf=to_lnm('station_1750-1755','station_1850-1855')
# odf.groupby(['word1','word2','period1','period2']).mean()

In [59]:
# odf=to_lnm('god_1750-1755','god_1850-1855')
# odf.groupby(['word1','word2','period1','period2']).mean()

### Scaling up

In [68]:
def lnm_postproc(odf):
    odf_ac=odf.reset_index().drop('run1',1).drop('run2',1)
    odx={
        'word1':odf_ac.iloc[0].word1,
        'word2':odf_ac.iloc[0].word2,
        'period1':odf_ac.iloc[0].period1,
        'period2':odf_ac.iloc[0].period2,
        **dict(odf_ac.mean()),
    }
    return odx

In [70]:
# lnm_postproc(odf)

In [72]:
def lnm_(objd):
    res=to_lnm(**objd)
    if res is None: res=pd.DataFrame()
    return res

def lnm(word_or_words1,word_or_words2=None,period_or_periods=None,k=25,num_proc=1,cache=True):
    
    ## input
    # words1?
    words1=tokenize_fast(word_or_words1) if type(word_or_words1)==str else list(word_or_words1)
    # words2?
    if word_or_words2 is None:
        words2=words1
    elif type(word_or_words2)==str:
        words2=tokenize_fast(word_or_words2)
    else:
        words2=list(word_or_words2)
    # periods?
    if period_or_periods is None:
        periods=get_default_periods()
    elif type(period_or_periods)==str:
        periods=tokenize_fast(period_or_periods)
    else:
        periods=list(period_or_periods)
        
    ## preproc
    objs = [
        dict(
            word1_period1=(w1,p1),
            word2_period2=(w2,p2),
            k=k,
            num_proc=1,
            cache=cache,
            progress=False
        )
        for p1 in periods
        for p2 in periods
        for w1,w2 in zip(words1,words2)
        if p1<p2 or (p1<=p2 and w1!=w2)
    ]
#     objs=objs[:10]#random.sample(objs,10)
    
    ## proc
    iterr=pmap_iter(
        lnm_,
        objs,
        num_proc=num_proc,
        desc='Measuring local neighb. distance across word-periods'
    )
    
    ## postproc
    o=[]
    for odf in iterr:
        odx=lnm_postproc(odf)
        if odx: o.append(odx)
    return pd.DataFrame(o).set_index(['word1','word2','period1','period2'])

In [73]:
dflnm = lnm('virtue',num_proc=4)

Measuring local neighb. distance across word-periods [x4]: 100%|██████████| 1128/1128 [05:36<00:00,  3.35it/s]


In [75]:
dflnm.sort_values('lnm')

lnm  mneighb_size  neighb1_size  \
word1  word2  period1   period2                                           
virtue virtue 1765-1770 1775-1780  0.015040          34.0          34.0   
              1885-1890 1890-1895  0.015813          32.0          32.0   
              1840-1845 1845-1850  0.016457          30.0          30.0   
              1920-1925 1925-1930  0.016968          34.0          34.0   
              1765-1770 1770-1775  0.017116          34.0          34.0   
...                                     ...           ...           ...   
              1795-1800 1930-1935  0.184179          43.1          44.0   
              1790-1795 1930-1935  0.188832          43.7          44.1   
              1780-1785 1930-1935  0.193601          42.4          43.4   
              1805-1810 1930-1935  0.195484          44.5          44.5   
              1785-1790 1930-1935  0.198604          43.7          44.1   

                                   neighb2_size  
word1  word2  period1   period2                  
virtue virtue 1765-1770 1775-1780          34.0  
              1885-1890 1890-1895          32.0  
              1840-1845 1845-1850          30.0  
              1920-1925 1925-1930          34.0  
              1765-1770 1770-1775          34.0  
...                                         ...  
              1795-1800 1930-1935          46.1  
              1790-1795 1930-1935          46.6  
              1780-1785 1930-1935          45.0  
              1805-1810 1930-1935          47.0  
              1785-1790 1930-1935          46.6  

[1128 rows x 4 columns]

In [ ]:
stop

In [26]:
dflnm = lnm('virtue',num_proc=4)

Measuring local neighb. distance across word-periods [x4]: 100%|██████████| 1128/1128 [06:34<00:00,  2.86it/s]


In [28]:
dflnm

lnm  mneighb_size  neighb1_size  \
word1  word2  period1   period2                                           
virtue virtue 1720-1725 1725-1730  0.045989         49.00          49.1   
                        1730-1735  0.089062         46.50          46.6   
                        1735-1740  0.088556         44.40          44.7   
                        1740-1745  0.091573         47.10          47.6   
                        1745-1750  0.110705         46.10          47.1   
...                                     ...           ...           ...   
              1940-1945 1950-1955  0.025960         39.00          39.0   
                        1955-1960  0.034473         42.26          42.3   
              1945-1950 1950-1955  0.024584         39.00          39.0   
                        1955-1960  0.034854         44.72          44.9   
              1950-1955 1955-1960  0.024161         40.16          40.2   

                                   neighb2_size     ttest_t        ttest_p  
word1  word2  period1   period2                                             
virtue virtue 1720-1725 1725-1730          49.1   76.269859  2.296732e-143  
                        1730-1735          46.6   76.578793  1.099767e-143  
                        1735-1740          44.7   76.907171  5.042976e-144  
                        1740-1745          47.6  103.154618  1.941106e-167  
                        1745-1750          47.1   74.666937  1.096989e-141  
...                                         ...         ...            ...  
              1940-1945 1950-1955          39.0   10.697155   3.663252e-21  
                        1955-1960          42.3   15.247627   1.068863e-34  
              1945-1950 1950-1955          39.0   11.901833   1.042592e-24  
                        1955-1960          44.9   23.380784   1.535831e-57  
              1950-1955 1955-1960          40.2   10.118316   1.731733e-19  

[1128 rows x 6 columns]

In [31]:
round(dflnm.sort_values('lnm'),2)

lnm  mneighb_size  neighb1_size  \
word1  word2  period1   period2                                       
virtue virtue 1765-1770 1770-1775  0.02          34.0          34.0   
                        1775-1780  0.02          34.0          34.0   
              1850-1855 1860-1865  0.02          31.0          31.0   
              1885-1890 1890-1895  0.02          32.0          32.0   
              1920-1925 1925-1930  0.02          34.0          34.0   
...                                 ...           ...           ...   
              1795-1800 1930-1935  0.18          43.1          44.0   
              1790-1795 1930-1935  0.19          43.7          44.1   
              1780-1785 1930-1935  0.19          42.4          43.4   
              1805-1810 1930-1935  0.20          44.5          44.5   
              1785-1790 1930-1935  0.20          43.7          44.1   

                                   neighb2_size  ttest_t  ttest_p  
word1  word2  period1   period2                                    
virtue virtue 1765-1770 1770-1775          34.0     7.56     0.00  
                        1775-1780          34.0     7.78     0.00  
              1850-1855 1860-1865          31.0     2.39     0.02  
              1885-1890 1890-1895          32.0     1.47     0.14  
              1920-1925 1925-1930          34.0     2.61     0.01  
...                                         ...      ...      ...  
              1795-1800 1930-1935          44.0    91.08     0.00  
              1790-1795 1930-1935          44.1    81.78     0.00  
              1780-1785 1930-1935          43.4    84.12     0.00  
              1805-1810 1930-1935          44.5    82.13     0.00  
              1785-1790 1930-1935          44.1    82.50     0.00  

[1128 rows x 6 columns]

In [76]:
round(dflnm.sort_values('lnm'),2)

lnm  mneighb_size  neighb1_size  \
word1  word2  period1   period2                                       
virtue virtue 1765-1770 1775-1780  0.02          34.0          34.0   
              1885-1890 1890-1895  0.02          32.0          32.0   
              1840-1845 1845-1850  0.02          30.0          30.0   
              1920-1925 1925-1930  0.02          34.0          34.0   
              1765-1770 1770-1775  0.02          34.0          34.0   
...                                 ...           ...           ...   
              1795-1800 1930-1935  0.18          43.1          44.0   
              1790-1795 1930-1935  0.19          43.7          44.1   
              1780-1785 1930-1935  0.19          42.4          43.4   
              1805-1810 1930-1935  0.20          44.5          44.5   
              1785-1790 1930-1935  0.20          43.7          44.1   

                                   neighb2_size  
word1  word2  period1   period2                  
virtue virtue 1765-1770 1775-1780          34.0  
              1885-1890 1890-1895          32.0  
              1840-1845 1845-1850          30.0  
              1920-1925 1925-1930          34.0  
              1765-1770 1770-1775          34.0  
...                                         ...  
              1795-1800 1930-1935          46.1  
              1790-1795 1930-1935          46.6  
              1780-1785 1930-1935          45.0  
              1805-1810 1930-1935          47.0  
              1785-1790 1930-1935          46.6  

[1128 rows x 4 columns]

In [47]:
dflnm.sort_values('lnm')

lnm  mneighb_size  neighb1_size  \
word1  word2  period1   period2                                           
virtue virtue 1765-1770 1770-1775  0.015550          34.0          34.0   
                        1775-1780  0.016004          34.0          34.0   
              1850-1855 1860-1865  0.017090          31.0          31.0   
              1885-1890 1890-1895  0.017718          32.0          32.0   
              1920-1925 1925-1930  0.017806          34.0          34.0   
...                                     ...           ...           ...   
              1795-1800 1930-1935  0.184336          43.1          44.0   
              1790-1795 1930-1935  0.188397          43.7          44.1   
              1780-1785 1930-1935  0.194371          42.4          43.4   
              1805-1810 1930-1935  0.195168          44.5          44.5   
              1785-1790 1930-1935  0.198876          43.7          44.1   

                                   neighb2_size    ttest_t        ttest_p  
word1  word2  period1   period2                                            
virtue virtue 1765-1770 1770-1775          34.0   7.558546   1.740287e-12  
                        1775-1780          34.0   7.781375   4.635320e-13  
              1850-1855 1860-1865          31.0   2.387131   1.797209e-02  
              1885-1890 1890-1895          32.0   1.465439   1.444716e-01  
              1920-1925 1925-1930          34.0   2.611502   9.742333e-03  
...                                         ...        ...            ...  
              1795-1800 1930-1935          44.0  91.078010  1.800135e-157  
              1790-1795 1930-1935          44.1  81.781969  6.757418e-149  
              1780-1785 1930-1935          43.4  84.123890  3.850951e-151  
              1805-1810 1930-1935          44.5  82.128245  3.119957e-149  
              1785-1790 1930-1935          44.1  82.503693  1.354408e-149  

[1128 rows x 6 columns]

In [60]:
# def lnm_postproc_thorough(odf):
#     odf=odf.reset_index().drop('run1',1).drop('run2',1)
#     odf['period_cmp']=['within' if x==y else 'across' for x,y in zip(odf.period1,odf.period2)]
#     odf_wi=odf[odf.period_cmp=='within']
#     odf_ac=odf[odf.period_cmp=='across']
#     lnm_wi=odf_wi.lnm
#     lnm_ac=odf_ac.lnm
#     ttd=ttest_ind(lnm_ac,lnm_wi)
#     odx={
#         'word1':odf_ac.iloc[0].word1,
#         'word2':odf_ac.iloc[0].word2,
#         'period1':odf_ac.iloc[0].period1,
#         'period2':odf_ac.iloc[0].period2,
        
#         **dict(odf_ac.mean()),
        
#         'ttest_t':ttd.statistic,
#         'ttest_p':ttd.pvalue
#     }
    
#     return odx

# def cdist_dfpiv(dfpiv):
#     o=[]
#     for i1 in dfpiv.index:
#         for i2 in dfpiv.index:
#             if i1<i2:
#                 dfcmp=dfpiv.loc[[i1,i2]]
                
#                 dfcmpd=dfcmp.dropna(axis=1)
                
#                 csim=fastdist.cosine(
#                     dfcmpd.iloc[0].values.astype(float),
#                     dfcmpd.iloc[1].values.astype(float)
#                 )
                
#                 odx={
#                     'i1':i1,
#                     'i2':i2,
#                     'cdist':1-csim,
#                     'size1':len(dfcmp.iloc[0].dropna()),
#                     'size2':len(dfcmp.iloc[1].dropna()),
#                     'size':len(dfcmpd.columns)
#                 }
#                 o.append(odx)
#     return pd.DataFrame(o)
            

In [38]:
# def to_lnm_thorough(word1_period1,word2_period2,k=25,num_proc=1,cache=True,progress=True):
    
#     word1,period1=word1_period1.split('_') if type(word1_period1)==str else word1_period1
#     word2,period2=word2_period2.split('_') if type(word2_period2)==str else word2_period2

#     qstr=f'{word1}_{period1},{word2}_{period2},k={k}'
#     if cache:
#         with get_veclib('lnm') as vl:
#             if qstr in vl:
#                 return vl[qstr]
    
#     dfnbr1=to_nbr(word1,period1,num_proc=num_proc,progress=progress)
#     dfnbr2=to_nbr(word2,period2,num_proc=num_proc,progress=progress)
    
#     # filter words in both models
#     dfmetanbr=dfnbr1.iloc[:k].append(dfnbr2.iloc[:k])
#     metaneighb=list(set(dfmetanbr.index.get_level_values('neighbor')))
    
#     # dists
#     dfcdist1=cdist(word1,period1,neighbors=metaneighb,progress=progress,num_proc=num_proc)
#     dfcdist2=cdist(word2,period2,neighbors=metaneighb,progress=progress,num_proc=num_proc)
#     dfcdist=dfcdist1.append(dfcdist2)
    
#     odf=dfcdist.reset_index()
#     odf['word_period_run']=list(zip(odf.word,odf.period,odf.run))
#     dfpiv=odf.pivot('word_period_run','neighbor','cdist')
    
#     dfpiv_dists = cdist_dfpiv(dfpiv)
#     o=[]
#     for i,row in dfpiv_dists.iterrows():
#         odx={
#             'word1':row.i1[0],
#             'word2':row.i2[0],
#             'period1':row.i1[1],
#             'period2':row.i2[1],
#             'run1':row.i1[2],
#             'run2':row.i2[2],
#             'lnm':row.cdist,
#             'mneighb_size':row['size'],
#             'neighb1_size':row.size1,
#             'neighb2_size':row.size2,
#         }
#         o+=[odx]
#     odf=pd.DataFrame(o)
#     index_cols=['word1','word2','period1','period2','run1','run2']
#     odf=odf.set_index(index_cols)
    
#     if cache:
#         with get_veclib('lnm',autocommit=True) as vl:
#             vl[qstr]=odf
    
#     return odf